<a href="https://colab.research.google.com/github/ImagingDataCommons/crdc_compound_objects/blob/main/CRDC_compound_object.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

This notebook investigates a compound object replacement for DRS bundles. The compound object is a JSON structure that is based on the DrsObject, but is intended to be extensible in support of various use cases. This is intended as a strawman proposal.

## Revisions
### 28 Oct
Removed fields...'size', 'created_time', 'updated_time' and 'version'...from the CRDC object. These are available in the DrsObjects that will be returned when the DRS URI in an access method is resolved.

# Compound objects

The DRS URI of an IDC DICOM instance can be resolved to obtain the location of the corresponding file object no matter where it is available. Similarly, it should be possible to resolve the DRS URL of an IDC DICOM series to obtain the location of the DICOM instances in that series. For this purpose we have defined a compound object, a JSON data structure, that contains DRS URLs that can be resolved to various embodiments of the corresponding series. We expect that the same structure can be used to represent other complex objects in other use cases.

The proposed compound object is based on the DrsObject, but with significant modifications. Rather than include a formal JSON schema, we will show an example that might be used to represent an IDC series.
```
{
  "encoding_version": "1.0",
  "description": "IDC CRDC DICOM series compound object",
  "object_type": "DICOM series",
  "id": "f5d6b517-2c02-4035-9444-0f15be7180ff",
  "name": "1.3.6.1.4.1.14519.5.2.1.8421.4018.242623218081388666714980416901",
  "self_uri": "drs://dg.4DFC/f5d6b517-2c02-4035-9444-0f15be7180ff",
  "access_methods": [
    {
      "access_method": "children",
      "mime_type": 'application/dicom'
      "description": "List of DRS URIs of instances in this series",
      "contents": [
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.8421.4018.218816382845480339633358956983",
          "drs_uri": "drs://dg.4DFC/01210a30-8395-498c-905f-6667db67101a"
        },
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.8421.4018.241858580971386059648748116938",
          "drs_uri": "drs://dg.4DFC/01b37013-7484-4968-8a9a-3a6b28c457e9"
        },
        :
        :
        :
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.8421.4018.306949952402324943719349396501",
          "drs_uri": "drs://dg.4DFC/fb8a3f58-3e6d-48c5-8ed7-08aac223213e"
        }
      ]
    },
    {
      "method": "folder_object",
      "mime_type": 'application/json'
      "description": "DRS URI that resolves to a gs or s3 folder object corresponding to this series",
      "contents": [
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.8421.4018.242623218081388666714980416901",
          "drs_uri": "drs://dg.4DFC/some_TBD_uuid"
        }
      ]
    },
    {
      "method": "archive_package",
      "mime_type": 'application/zip'
      "description": "DRS URI that resolves to a zip archive of the instances in this series",
      "contents": [
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.8421.4018.242623218081388666714980416901.zip",
          "drs_uri": "drs://dg.4DFC/some_TBD_uuid"
        }
      ]
    }
  ],
  "aliases": []
}
```

The `description`, `id`, `name`, `self_uri`, `size`, `created_time`, `updated_time`, and `version` fields are as defined in the DrsObject. The `encoding_version` field is added in the expectation that this JSON structure might need revision.

When used to represent an IDC DICOM series, the value of `id` would be the the `series_uuid` (A UUID4 assigned to a version of a DICOM series) of the series, and the value of `name` would be the DICOM SOPInstanceUID of the series.

The `access_methods` field, while having the same name as a field in the DrsObject, is very different. The `access_methods` recognizes that there may be more than one 'embodiment' or method of accessing the same object.

In the case of an DICOM series in the IDC data set, the series might be embodied as the set of GCS (and, at some future time, S3) DICOM instance blobs and accessed by resolving a DRS URI for each such instance. Another possible/expected method is as an archive such as a zip, again embodied as a GCS or S3 blob. A third such method could be as a GCS or S3 'directory' that contains the instances in the series.

Rather than define an IDC specific compound object for this purpose, we have tried to specify a JSON encoding that is somewhat self describing and adaptable to other use cases. Admittedly, the goal of supporting arbitrary access types requires some use case specific knowledge on the part of the user. Specifically, in the case of the IDC DICOM series compound object, the user must know that they are looking for a 'children' method and/or a 'folder_object' method and/or a 'package' method. That is, other methods might be defined in support of other use cases, and which have different `method` IDs.

Each access_method has:
*  an `access_method`, a string that identifies the method,
*  a `mime_type`, the mime_type of the data element(s) to which each DRS URI in the `contents` array resolves 
*  a `description`, an arbitrary use-case specific string 
*  a `contents` array, each element of which is 
 * an `id`, an arbitrary use-case specific string, 
 * a `drs_uri`, a DRS URI that can be resolved to some object.

At a very high level, the expectation is that, having obtained such a compound object, the user will find the access_method of interest by searching for some `access_method` string. Having found such an access method, the user will proceed to select one or more elements from the contents array, perhaps based on the `id` value, and then proceed to resolve the corresponding DRS_URI. In subsequent sections, we'll demonstrate this process against the above example compound object.

Note that we will refer to the CRDC compound object of an IDC DICOM series as a `series object`. 

# Demo
We've populated a GCS bucket, `cdrcobj`, with a selected set of series from the NLST collection.

Instances have names like `<series_uuid>/<instance_uuid>.dcm` as described in the [Hierarchical Instance Names](hhttps://colab.research.google.com/drive/1jPq_VwwdnRyT-S47hoozTxm396G5mutn#scrollTo=ZB63N50uGLXK&uniqifier=1ttps://) notebook.

## Demo config

In [1]:
demo_bucket = 'crdcobj'

The jq JSON processor is useful for inspecting JSON:

In [4]:
!apt -qq update
!apt -qq install jq

30 packages can be upgraded. Run 'apt list --upgradable' to see them.
jq is already the newest version (1.5+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In order to access GCS, we first authenticate to Colab:

In [5]:
from google.colab import auth
auth.authenticate_user()

We'll be using the GCS libraries and the Requests modul

In [6]:
import os
import shutil
import json
from google.cloud import storage
from google.cloud.storage import Blob
import requests
client = storage.Client()
bucket = client.bucket(demo_bucket)

We have also created a series object like the example above for each series in the demo dataset. Each has a GCS URL like:
`gs://crdcobj/<series_uuid>/crdcobj.json`
We might have obtained such a GCS URL from an IDC cohort manifest or from a BQ query. In that case we can get the series object using gsutil or other tools. We'll assume that we have obtain the GCS URL of the compound object for the series having <series_uuid>:  
`f5d6b517-2c02-4035-9444-0f15be7180ff/crdcobj.json`:



In [7]:
!gsutil cat gs://crdcobj/f5d6b517-2c02-4035-9444-0f15be7180ff/crdcobj.json | jq

{
  "encoding_version": "1.0",
  "description": "IDC CRDC DICOM series compound object",
  "object_type": "DICOM series",
  "id": "f5d6b517-2c02-4035-9444-0f15be7180ff",
  "name": "1.3.6.1.4.1.14519.5.2.1.7009.9004.330787529051851659409647497231",
  "self_uri": "drs://dg.4DFC/f5d6b517-2c02-4035-9444-0f15be7180ff",
  "access_methods": [
    {
      "method": "children",
      "mime_type": "application/dicom",
      "description": "List of DRS URIs of instances in this series",
      "contents": [
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.7009.9004.556887014045436659051172668458",
          "drs_uri": "drs://dg.4DFC/65965c28-adc2-4d2e-811c-5eabae295153"
        },
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.7009.9004.296554997667882869766648987541",
          "drs_uri": "drs://dg.4DFC/6989ff3e-a0be-4bd2-946a-d225e5211de0"
        },
        {
          "name": "1.3.6.1.4.1.14519.5.2.1.7009.9004.309634580422520146136818348640",
          "drs_uri": "drs://dg.4DFC/970f4

We'll use this example throughout this notebook.

## Using the instance name hierarchy

The Hierarchical Instance Names notebook describes accessing the instances in a series when you know the `folder object` of the series. If we have a `series object`, we can obtain the GCS URL of its folder object:

1. Resolve the DRS URI in the 'folder_object' method in the series's compound object (shown as `drs://dg.4DFC/some_TBD_uuid' in the above ecxample...we've not yet generated these UUIDs.)
2. Resolving the DRS URI will return a DrsObject. An `AccessURL` in the DrsObject will contain a `url` whose value is the GCS URL of the folder object.

We cannot demonstrate this because we have not generated DRS URIs for folder objects.

## Obtaining Unsigned or Signed URLs for DICOM Instances
The Hierarchical Instance Names notebook describes various methods for accessing the instances in a series based on hierarchical naming. This access method can only be used when access to the instance data does not require a signed URL. 

Currently all IDC data is accessible using an unsigned URL, but, in the future, access to some data might require a signed license. For this purpse, the `series object` includes a `children` access method.

For demonstration purposes, we will assume that we have obtained the GCS URL, `gs://whc_sjo/f5d6b517-2c02-4035-9444-0f15be7180ff/crdcobj.json` of the previously discussed `series object`:



In [8]:
series_object_gcs_url = 'gs://crdcobj/f5d6b517-2c02-4035-9444-0f15be7180ff/crdcobj.json'

Next we'll use the GCS API to get the contents of that folder object:

In [9]:
# Create a bucket object
bucket = client.get_bucket(series_object_gcs_url.split('/')[2])
#Note that in the following we explicitly append '/' to the blob name 
src_blob = Blob(f"{series_object_gcs_url.split('/',3)[3]}", bucket)
series_object = json.loads(src_blob.download_as_string())
series_object

{'encoding_version': '1.0',
 'description': 'IDC CRDC DICOM series compound object',
 'object_type': 'DICOM series',
 'id': 'f5d6b517-2c02-4035-9444-0f15be7180ff',
 'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.330787529051851659409647497231',
 'self_uri': 'drs://dg.4DFC/f5d6b517-2c02-4035-9444-0f15be7180ff',
 'access_methods': [{'method': 'children',
   'mime_type': 'application/dicom',
   'description': 'List of DRS URIs of instances in this series',
   'contents': [{'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.556887014045436659051172668458',
     'drs_uri': 'drs://dg.4DFC/65965c28-adc2-4d2e-811c-5eabae295153'},
    {'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.296554997667882869766648987541',
     'drs_uri': 'drs://dg.4DFC/6989ff3e-a0be-4bd2-946a-d225e5211de0'},
    {'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.309634580422520146136818348640',
     'drs_uri': 'drs://dg.4DFC/970f413b-a7b6-4018-8e38-fb0e4c403463'},
    {'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.322338755286007402083501595731

Now we can resolve each DRS URI to obtain a signed URL, and then access the corresponding blob. We'll first make a list of the instance DRS URIs

In [10]:
# Try to find a 'children' access method
method = next(item for item in series_object['access_methods'] if item['method'] == 'children')
if method:
  drs_ids = [content['drs_uri'].split('drs://')[1] for content in method['contents']]
  print(drs_ids)
else:
  print('A "children" access method was not found')



['dg.4DFC/65965c28-adc2-4d2e-811c-5eabae295153', 'dg.4DFC/6989ff3e-a0be-4bd2-946a-d225e5211de0', 'dg.4DFC/970f413b-a7b6-4018-8e38-fb0e4c403463', 'dg.4DFC/31c0614f-5906-402c-9334-90322c064b9d', 'dg.4DFC/fd1f6002-98f5-42ac-a6dd-ecfdb65d65b5', 'dg.4DFC/b2136300-e6a6-4bbf-a9fc-f3f7ad6cbd5e', 'dg.4DFC/88bed033-949c-4546-86a6-c6f7ed6238d6', 'dg.4DFC/6e33bc0e-c46d-40a3-ae9d-08c49bbaa0b8', 'dg.4DFC/011bc2f0-3b70-4a9d-b9de-9b9022c6facc', 'dg.4DFC/31ac2cc2-64f6-4274-b4fb-a314a9275784', 'dg.4DFC/51c04034-41fc-4cef-943d-9b565e3b1f69', 'dg.4DFC/23413b86-f444-4149-8cdf-dd5f1129350c', 'dg.4DFC/7f1d4762-74ff-4337-a3a1-b1d192a78713', 'dg.4DFC/698f4f8f-d1fe-45cf-a94a-9909f4090531', 'dg.4DFC/732d99f3-99b6-4281-aabf-e990fe7cb870', 'dg.4DFC/1bbe058f-faf7-4bbc-83b9-0f8fc953b2d5', 'dg.4DFC/e16f04be-e186-49b6-b522-103dded12487', 'dg.4DFC/49c381eb-8c83-4f66-8291-b248bed8be4d', 'dg.4DFC/e3a2afa0-ac54-4608-9e1c-75533a6b6408', 'dg.4DFC/dd7eaba3-2b5a-4244-b4cc-7fbadeceebe8', 'dg.4DFC/b9117665-c584-4073-a049-41434f

We can resolve each DRS ID to get the GCS URL of the instance blob and copy each blob to the local file system.

Warning: This operation is very slow.

In [ ]:
drs_server = 'https://nci-crdc.datacommons.io/ga4gh/drs/v1/objects'
!mkdir -vp /tmp/dicom_data

for drs_id in drs_ids:
  url = f"{drs_server}/{drs_id}"
  result = requests.get(url)
  result.raise_for_status()
  drsObject = result.json()
  # We need the access_id to get the signed URL 
  access_id = drsObject['access_methods'][0]['access_id']
  url = f'{url}/access/{access_id}'
  # Get the signed URL
  result = requests.get(url)
  result.raise_for_status()
  signed_url = result.json()['url']

  # Use requests to get the blob contents
  instance_blob_id = drs_id.split('/')[-1]
  with open(f"/tmp/dicom_data/{instance_blob_id}", "wb") as file_obj:
    result = requests.get(signed_url)
    file_obj.write(result.content)
  print(f'resolved and copied {drs_id}')

# Demonstrate that we've downloaded the blobs.      
!ls -l /tmp/dicom_data

# Clean up
!rm -vr /tmp/dicom_data





Of course, if the instance data in a series can be accessed using unsigned URLs, we can use this same process, but just skipping the extra step to get a signed URL.

We'll use the GCS library to copy the blobs.

In [ ]:
drs_server = 'https://nci-crdc.datacommons.io/ga4gh/drs/v1/objects'
gcs_server = f'https://storage.googleapis.com'
!mkdir -vp /tmp/dicom_data

for drs_id in drs_ids:
  url = f"{drs_server}/{drs_id}"
  result = requests.get(url)
  result.raise_for_status()
  drsObject = result.json()
  # We need the access_id to get the signed URL 
  unsigned_url = drsObject['access_methods'][0]['access_url']['url']

  # Use requests to get the blob contents
  bucket_id = unsigned_url.split('/')[2]
  instance_blob_id = unsigned_url.split('/')[-1]
  bucket = client.bucket(bucket_id)
  with open(f"/tmp/dicom_data/{instance_blob_id}", "wb") as file_obj:
     bucket.blob(instance_blob_id).download_to_file(file_obj)
  print(f'resolved and copied {drs_id}')

# Demonstrate that we've downloaded the blobs.      
!ls -l /tmp/dicom_data

# Clean up
!rm -vr /tmp/dicom_data

## Obtaining Unsigned or Signed URLs of an Archive
As mentioned, there might be an archive, such as a zip, of all the instances in some series. Such a blob might be freely accessible or might require a signed URL.

For demonstration purposes, we will assume that we have obtained the GCS URL, `gs://whc_sjo/f5d6b517-2c02-4035-9444-0f15be7180ff/crdcobj.json` of the previously discussed `series object`:

In [ ]:
series_object_gcs_url = 'gs://crdcobj/f5d6b517-2c02-4035-9444-0f15be7180ff/crdcobj.json'

Next we'll use the GCS API to get the contents of series  object:

In [13]:
# Create a bucket object
bucket = client.get_bucket(series_object_gcs_url.split('/')[2])
#Note that in the following we explicitly append '/' to the blob name 
src_blob = Blob(f"{series_object_gcs_url.split('/',3)[3]}", bucket)
series_object = json.loads(src_blob.download_as_string())
series_object

{'encoding_version': '1.0',
 'description': 'IDC CRDC DICOM series compound object',
 'object_type': 'DICOM series',
 'id': 'f5d6b517-2c02-4035-9444-0f15be7180ff',
 'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.330787529051851659409647497231',
 'self_uri': 'drs://dg.4DFC/f5d6b517-2c02-4035-9444-0f15be7180ff',
 'access_methods': [{'method': 'children',
   'mime_type': 'application/dicom',
   'description': 'List of DRS URIs of instances in this series',
   'contents': [{'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.556887014045436659051172668458',
     'drs_uri': 'drs://dg.4DFC/65965c28-adc2-4d2e-811c-5eabae295153'},
    {'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.296554997667882869766648987541',
     'drs_uri': 'drs://dg.4DFC/6989ff3e-a0be-4bd2-946a-d225e5211de0'},
    {'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.309634580422520146136818348640',
     'drs_uri': 'drs://dg.4DFC/970f413b-a7b6-4018-8e38-fb0e4c403463'},
    {'name': '1.3.6.1.4.1.14519.5.2.1.7009.9004.322338755286007402083501595731

Get the `archive_package` access method from the series object:

In [16]:

method = next(item for item in series_object['access_methods'] if item['method'] == 'archive_package')
if method:
  drs_uri = method['contents'][0]['drs_uri']
  print(f'drs_uri: {drs_uri}')
else:
  print('A "archive_package" access method was not found')

drs_uri: drs://dg.4DFC/some_TBD_uuid


Note that we have not yet created any such archives and therefore can't demonstrate resolving the DRS URI of such an object. However, the process is as for any other DRS URI.

Similarly, if the `series object` has a `folder_object` method, we can resolve its DRS URI to obtain an unsigned or signed URL.

In [17]:
method = next(item for item in series_object['access_methods'] if item['method'] == 'folder_object')
if method:
  drs_uri = method['contents'][0]['drs_uri']
  print(f'drs_uri: {drs_uri}')
else:
  print('A "folder_object" access method was not found')

drs_uri: drs://dg.4DFC/some_TBD_uuid


We have not indexed the folders in this example so cannot demonstrate a resolving such a DRS URI.